In [0]:
!apt-get install python3-dev default-libmysqlclient-dev
!pip install mysqlclient

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
!pip install pyunpack
!pip install patool rtree
!pip install graphviz
!apt-get install -y libspatialindex-dev
!apt install gdal-bin python-gdal python3-gdal 
!apt install python3-rtree 
!pip install geopandas rtree fiona shapely pyproj descartes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from pyunpack import Archive
import patoolib
# !wget 'http://www.bangkokgis.com/bangkokgis_2008/userfiles/files/download/shapefile/administration/BMASubDistrict_Line.rar'
!mkdir 'BMASubDistrict_Line'
Archive('BMASubDistrict_Line.rar').extractall('BMASubDistrict_Line')

In [0]:
!ls BMASubDistrict_Line

ADMIN_LINE.cpg	ADMIN_LINE.prj	ADMIN_LINE.sbx	ADMIN_LINE.shp.xml
ADMIN_LINE.dbf	ADMIN_LINE.sbn	ADMIN_LINE.shp	ADMIN_LINE.shx


In [0]:
import geopandas

gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.lat,df.long))

# point = geopandas.GeoDataFrame.from_file('points.shp') 
poly  = geopandas.GeoDataFrame.from_file('BMASubDistrict_Line/ADMIN_LINE.shp')
from geopandas.tools import sjoin
pointInPolys = sjoin(gdf, poly, how='left')
# grouped = pointInPolys.groupby('index_right')

/usr/local/lib/python3.6/dist-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:32647)
  "(%s != %s)" % (left_df.crs, right_df.crs)


### Notice the appended column on the right with the subdistrict

In [0]:
pointInPolys

,index,address.coordinates.coordinates,address.coordinates.type,condo.common_fee,condo.id,condo.parking_slot_amount,condo.parking_slot_percentage,condo.unit_amount,condo_name,detail.bathroom_amount,detail.bedroom_amount,detail.floor,detail.otherroom_amount,detail.parking_slot,full_address,new_condo_id,price_per_sqm,price_rent,price_sell,room_size,lat,long
0,0,"[100.57029591131, 13.731550581483]",Point,90,1419,160,102,156,The Diplomat 39,1,1,8,0,0,ซอย สุขุมวิท 39 แขวงคลองเตยเหนือ เขตวัฒนา กรุง...,1419,280000,NaN,16790000,60.00,100.570296,13.731551
1,1,"[100.55620782546, 13.806206734004]",Point,75,147,420,50,841,The Line จตุจักร – หมอชิต,2,3,28,0,0,ถนนพหลโยธิน แขวงจอมพล เขตจตุจักร กรุงเทพมหานคร...,147,216000,NaN,16800000,77.75,100.556208,13.806207
2,2,"[100.566444, 13.743262]",Point,48,1436,55,74,74,VOQUE สุขุมวิท 31,2,2,None,0,0,ซอยสุขุมวิท 31 แขวงคลองเตยเหนือ เขตวัฒนา กรุงเ...,1436,139000,NaN,16900000,122.00,100.566444,13.743262
3,3,"[100.58022754954, 13.72258985005]",Point,0,554,0,0,0,สิริ แอท สุขุมวิท,2,2,24,0,0,ซอยสุขุมวิท 38-40 แขวงพระโขนง เขตคลองเตย กรุงเ...,554,235000,NaN,16900000,72.00,100.580228,13.722590
4,4,"[100.51066044233, 13.72752828353]",Point,0,495,0,100,379,แมกโนเลียส์ วอเตอร์ฟรอนท์ เรสซิเดนซ์ ณ ไอคอนสย...,1,1,None,0,0,ถนนเจริญนคร แขวงคลองต้นไทร เขตคลองสาน กรุงเทพม...,495,256000,NaN,16900000,65.99,100.510660,13.727528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,3883,"[100.45212164425, 13.768441754531]",Point,0,2176,0,0,0,ป. ปิ่นเกล้า คอนโดมิเนียม 2,1,1,5,1,0,ซอยจรัญ 35 ถนนแก้วเงินทอง แขวงคลองชักพระ เขตตล...,2176,16000,NaN,390000,25.00,100.452122,13.768442
3884,3884,"[100.60925417006, 13.987822825648]",Point,0,254,0,0,0,ฟิวเจอร์ เพลสคอนโด,1,Studio,14,0,0,ถนนรังสิต-ปทุมธานี ตำบลประชาธิปัตย์ อำเภอธัญบุ...,254,16000,NaN,450000,28.00,100.609254,13.987823
3885,3885,"[100.6397713, 13.761899]",Point,0,540,0,46,682,ไนท์บริดจ์ คอลลาจ รามคำแหง (Knightsbridge Coll...,1,1,None,1,0,ถนนรามคำแหง แขวงหัวหมาก เขตบางกะปิ กรุงเทพมหาน...,540,17000,NaN,450000,26.40,100.639771,13.761899
3886,3886,"[100.60925417006, 13.987822825648]",Point,0,254,0,0,0,ฟิวเจอร์ เพลสคอนโด,1,1,11,0,0,ถนนรังสิต-ปทุมธานี ตำบลประชาธิปัตย์ อำเภอธัญบุ...,254,17000,NaN,460000,27.00,100.609254,13.987823


### Next will be the model fitting.

In [0]:
from statsmodels.formula.api import ols
dfx=df.rename(columns={"detail.bathroom_amount": "bathroom_amount","detail.bedroom_amount":"bedroom_amount","detail.otherroom_amount":"otherroom_amount"})
model = ols('price_sell ~ (bathroom_amount) +	C(bedroom_amount) + (otherroom_amount) + room_size', data=dfx)
fitted_model = model.fit()
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price_sell   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     693.7
Date:                Sun, 15 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:31:17   Log-Likelihood:                -62750.
No. Observations:                3888   AIC:                         1.255e+05
Df Residuals:                    3880   BIC:                         1.256e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                   -3.657e+05   1.74e+05     -2.098      0.036   -7.07e+05   -2.39e+04
C(bedroom_amount)[T.2]      -3.279e+05   1.56e+05     -2.097      0.036   -6.34e+05   -2.13e+04
C(bedroom_amount)[T.3]      -3.722e+06   4.02e+05     -9.247      0.000   -4.51e+06   -2.93e+06
C(bedroom_amount)[T.4]      -1.754e+07   2.55e+06     -6.882      0.000   -2.25e+07   -1.25e+07
C(bedroom_amount)[T.Studio] -1.387e+06   1.57e+05     -8.809      0.000    -1.7e+06   -1.08e+06
bathroom_amount              1.007e+06   1.74e+05      5.796      0.000    6.66e+05    1.35e+06
otherroom_amount            -1.515e+06   8.07e+04    -18.779      0.000   -1.67e+06   -1.36e+06
room_size                    1.125e+05   3059.632     36.762      0.000    1.06e+05    1.18e+05
==============================================================================
Omnibus:                      543.871   Durbin-Watson:                   0.698
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1766.343
Skew:                           0.707   Prob(JB):                         0.00
Kurtosis:                       5.984   Cond. No.                     3.20e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Model 1
The model is in overall significant with the P value of the F-test lesser than 0.05. The variation is of the selling price is explained by the variables by 55.6%. All the variables yield statistically significant in thie model with the P value lesser than 0.05.

In [0]:
from statsmodels.formula.api import ols
dfx=df.rename(columns={"detail.bathroom_amount": "bathroom_amount","detail.bedroom_amount":"bedroom_amount"})
# model = ols('price_sell ~ C(bathroom_amount) +	C(bedroom_amount) + room_size', data=dfx)
model = ols('price_sell ~ C(SUBDISTRICT) +	room_size', data=dfx)
fitted_model = model.fit()
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price_sell   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     66.42
Date:                Sun, 15 Mar 2020   Prob (F-statistic):               0.00
Time:                        10:11:39   Log-Likelihood:                -53257.
No. Observations:                3338   AIC:                         1.067e+05
Df Residuals:                    3223   BIC:                         1.074e+05
Df Model:                         114                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -1.297e+06   1.48e+06     -0.876      0.381    -4.2e+06    1.61e+06
C(SUBDISTRICT)[T.100401.0]  4.153e+06   1.63e+06      2.540      0.011    9.47e+05    7.36e+06
C(SUBDISTRICT)[T.100402.0]  5.994e+06    1.5e+06      3.987      0.000    3.05e+06    8.94e+06
C(SUBDISTRICT)[T.100403.0]  5.917e+06   1.81e+06      3.267      0.001    2.37e+06    9.47e+06
C(SUBDISTRICT)[T.100404.0]   5.89e+06   1.91e+06      3.085      0.002    2.15e+06    9.63e+06
C(SUBDISTRICT)[T.100405.0]  6.174e+06   1.56e+06      3.949      0.000    3.11e+06    9.24e+06
C(SUBDISTRICT)[T.100502.0]  6.323e+05   1.53e+06      0.412      0.680   -2.38e+06    3.64e+06
C(SUBDISTRICT)[T.100508.0]   3.55e+05   1.68e+06      0.212      0.832   -2.93e+06    3.64e+06
C(SUBDISTRICT)[T.100601.0]  5.258e+05   1.56e+06      0.336      0.737   -2.54e+06    3.59e+06
C(SUBDISTRICT)[T.100608.0]  4.346e+05   1.51e+06      0.288      0.773   -2.52e+06    3.39e+06
C(SUBDISTRICT)[T.100701.0]  4.962e+06   1.71e+06      2.906      0.004    1.61e+06    8.31e+06
C(SUBDISTRICT)[T.100702.0]  4.467e+06   1.68e+06      2.664      0.008    1.18e+06    7.75e+06
C(SUBDISTRICT)[T.100703.0]  6.522e+06   2.56e+06      2.543      0.011    1.49e+06    1.16e+07
C(SUBDISTRICT)[T.100704.0]  5.663e+06   1.51e+06      3.744      0.000     2.7e+06    8.63e+06
C(SUBDISTRICT)[T.100905.0]  1.533e+06   1.51e+06      1.017      0.309   -1.42e+06    4.49e+06
C(SUBDISTRICT)[T.100910.0]  1.819e+06    1.5e+06      1.212      0.226   -1.12e+06    4.76e+06
C(SUBDISTRICT)[T.101001.0]   3.24e+05   1.68e+06      0.193      0.847   -2.96e+06    3.61e+06
C(SUBDISTRICT)[T.101101.0]  2.275e+05   1.65e+06      0.138      0.891   -3.01e+06    3.47e+06
C(SUBDISTRICT)[T.101103.0]  3.716e+05   1.68e+06      0.222      0.825   -2.92e+06    3.66e+06
C(SUBDISTRICT)[T.101203.0]  1.072e+06   1.52e+06      0.706      0.480   -1.91e+06    4.05e+06
C(SUBDISTRICT)[T.101204.0]  2.212e+06   1.57e+06      1.410      0.159   -8.63e+05    5.29e+06
C(SUBDISTRICT)[T.101301.0]  7.702e+05   2.56e+06      0.301      0.764   -4.25e+06    5.79e+06
C(SUBDISTRICT)[T.101303.0]  7.591e+04   2.56e+06      0.030      0.976   -4.95e+06     5.1e+06
C(SUBDISTRICT)[T.101401.0]  2.004e+06   1.54e+06      1.300      0.194   -1.02e+06    5.03e+06
C(SUBDISTRICT)[T.101406.0]  3.269e+06   1.53e+06      2.133      0.033    2.64e+05    6.27e+06
C(SUBDISTRICT)[T.101502.0]  7.623e+05   2.56e+06      0.298      0.766   -4.26e+06    5.78e+06
C(SUBDISTRICT)[T.101503.0]  7.936e+05   2.09e+06      0.380      0.704   -3.31e+06    4.89e+06
C(SUBDISTRICT)[T.101504.0]   1.16e+06   1.53e+06      0.758      0.449   -1.84e+06    4.16e+06
C(SUBDISTRICT)[T.101505.0]  4.033e+05   1.75e+06      0.230      0.818   -3.03e+06    3.83e+06
C(SUBDISTRICT)[T.101506.0]   1.18e+06   1.5

# Model 2
This model includes the subdisctict dummy variable. I hyphothesized that the price would be based on the area of the condominium. The results shows much higher R squares compared to Model 1. The F statistics is sound however, the dummary variables are mostly insignificant. This is probably due to lack of samples in various areas. 

# Summary
Regression is a popular model for determining house prices. Many research have proposed other additional variables such as distance to public transport, commute time, etc. as well as other models such as Regressor Decision Trees which are more sophisticated. With the aid of other tools in spatial analysis that derives more variables and selection of other models that require tuning would significantly improve analysis.